# https://www.sec.gov/Archives/edgar/full-index/2017/

# Current VER

In [ ]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup as soup
import lxml
import re
import os
import concurrent.futures
import requests
import os
import errno
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# DEFAULT_DATA_PATH = 'sec_10k/'
# DEFAULT_DATA_PATH = 'C:/Users/Rocku/Downloads/sec_10k/'
DEFAULT_DATA_PATH = 'D:/sec_10k/'

class SecCrawler():
    ua = UserAgent()
    failed_list = []
    
    def __init__(self):
        print("Path of the directory: " + DEFAULT_DATA_PATH)
        
    def create_document_list(self, data):
        soup_obj = BeautifulSoup(data, 'lxml')
        link_list = list()

        # If the link is .htm convert it to .html
        for link in soup_obj.find_all('filinghref'):
            url = link.string
            if link.string.split(".")[len(link.string.split("."))-1] == "htm":
                url += "l"
            link_list.append(url)
        link_list_final = link_list

        doc_list = list()
        doc_name_list = list()
        for k in range(len(link_list_final[:1])):
            required_url = link_list_final[0].replace('-index.html', '')
            txtdoc = required_url + ".txt"
            docname = txtdoc.split("/")[-1]
            doc_list.append(txtdoc)
            doc_name_list.append(docname)
        return doc_list, doc_name_list
    
    
    def save_in_directory(self, company_code, cik, priorto, doc_list, doc_name_list, filing_type):
        for j in range(len(doc_list)):
            base_url = doc_list[j]
            r = requests.get(base_url, headers={'User-Agent' : self.ua.random})
            report = soup(r.content.decode('utf-8'), 'lxml').get_text()
            report = re.sub(r"\s\s+", " ", report)
            report = re.sub(r"<.*?>", "", report)
            splitted = re.split(r'item\s*\d*\w+\W*\n+', report.lower())
            filename = '--'.join([company_code,cik,priorto,doc_name_list[j]])
            print(len(splitted))
            flag_write = True
            for chunk in splitted:
                if (chunk.lower().strip().startswith('business') and len(chunk) > 1000 and flag_write):
                    with open(DEFAULT_DATA_PATH + filename, "ab") as f:
                        f.write(chunk.encode('ascii', 'ignore'))
                    flag_write = False
                    print(filename, ' downloaded')
            
    def filing_10K(self, company_code, cik, priorto, count):
        base_url = "http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+str(cik)+"&type=10-K&dateb="+str(priorto)+"&owner=exclude&output=xml&count="+str(count)
        print(base_url)
        r = requests.get(base_url, headers={'User-Agent' : self.ua.random})
        data = r.text
        doc_list, doc_name_list = self.create_document_list(data)

        try:
            self.save_in_directory(company_code, cik, priorto, doc_list, doc_name_list, '10-K')
        except Exception as e:
            print(str(e))
            self.failed_list.append([company_code, cik, priorto, doc_list, doc_name_list])
            
        
def get_10k_path(year):
    ua = UserAgent()
    qtrs = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
#     qtrs = ['QTR1']  # Only consider 4th Querter of year
    columns = ['CIK', 'CompanyName', 'FormType', 'Date', 'FileName']
    base_url = "http://www.sec.gov/Archives/edgar/full-index/%s/" % year
    df_base = pd.DataFrame(columns=columns)
    
    for qtr in qtrs:
        headers = {'User-Agent' : ua.random}
        url = base_url + qtr + '/master.idx'
        print(url)
        lines = requests.get(url, headers=headers).text.splitlines()
        records = [tuple(line.split('|')) for line in lines[11:]]
        df = pd.DataFrame(records, columns=columns)
        TenK = df[df['FormType'] == '10-K']
        df_base = df_base.append(TenK)
        del df, TenK, records
    print('Total number of 10-K filings : ', len(df_base))
    return df_base

In [ ]:
if not os.path.exists(DEFAULT_DATA_PATH):
    os.makedirs(DEFAULT_DATA_PATH)

# Get urls of 10-K filings in 2017
idx_data = get_10k_path(2017)
idx_data.reset_index(inplace=True)

In [8]:
secCrawler = SecCrawler()
for ind,ex in idx_data.iterrows():
    CompanyName = re.sub(r"\W+", " ", ex['CompanyName']).strip()
    CIK = ex['CIK']
    Date = ex['Date'].replace('-','')
    secCrawler.filing_10K(CompanyName, CIK, Date, '1')

print('Total number of 10-K crawled : ', len(secCrawler.failed_list))

Path of the directory: D:/sec_10k/
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000209&type=10-K&dateb=20170314&owner=exclude&output=xml&count=1
18
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000228&type=10-K&dateb=20170221&owner=exclude&output=xml&count=1
2
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000229&type=10-K&dateb=20170210&owner=exclude&output=xml&count=1
24
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000232&type=10-K&dateb=20170330&owner=exclude&output=xml&count=1
22
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000298&type=10-K&dateb=20170309&owner=exclude&output=xml&count=1
31
IMPAC MORTGAGE HOLDINGS INC--1000298--20170309--0001558370-17-001542.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1000623&type=10-K&dateb=20170224&owner=exclude&output=xml&count=1
32
SCHWEITZER MAUDUIT INTERNATIONAL INC--1000623--20170224--0001000623-17-000036.txt  downloaded
h

2
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1008654&type=10-K&dateb=20170228&owner=exclude&output=xml&count=1
32
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1008848&type=10-K&dateb=20170227&owner=exclude&output=xml&count=1
13
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=100885&type=10-K&dateb=20170203&owner=exclude&output=xml&count=1
7
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1009672&type=10-K&dateb=20170301&owner=exclude&output=xml&count=1
32
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1009829&type=10-K&dateb=20170316&owner=exclude&output=xml&count=1
78
JAKKS PACIFIC INC--1009829--20170316--0001157523-17-000818.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1010086&type=10-K&dateb=20170307&owner=exclude&output=xml&count=1
22
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1010612&type=10-K&dateb=20170301&owner=exclude&output=xml&count=1
33
SYKES EN

49
MARATHON OIL CORP--101778--20170224--0000101778-17-000046.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1017793&type=10-K&dateb=20170310&owner=exclude&output=xml&count=1
19
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1017907&type=10-K&dateb=20170314&owner=exclude&output=xml&count=1
26
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1017968&type=10-K&dateb=20170315&owner=exclude&output=xml&count=1
45
EDGEWATER TECHNOLOGY INC DE--1017968--20170315--0001193125-17-083628.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1018164&type=10-K&dateb=20170316&owner=exclude&output=xml&count=1
21
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1018254&type=10-K&dateb=20170221&owner=exclude&output=xml&count=1
43
UNITED DOMINION REALTY L P--1018254--20170221--0000074208-17-000030.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1018281&type=10-K&dateb=20170117&owne

3
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1025378&type=10-K&dateb=20170224&owner=exclude&output=xml&count=1
25
W P Carey Inc--1025378--20170224--0001025378-17-000016.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=10254&type=10-K&dateb=20170315&owner=exclude&output=xml&count=1
15
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1025771&type=10-K&dateb=20170228&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1025835&type=10-K&dateb=20170224&owner=exclude&output=xml&count=1
46
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1025877&type=10-K&dateb=20170329&owner=exclude&output=xml&count=1
12
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1025996&type=10-K&dateb=20170215&owner=exclude&output=xml&count=1
27
KILROY REALTY CORP--1025996--20170215--0001025996-17-000049.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=102621

8
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035216&type=10-K&dateb=20170227&owner=exclude&output=xml&count=1
27
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035267&type=10-K&dateb=20170206&owner=exclude&output=xml&count=1
20
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035443&type=10-K&dateb=20170131&owner=exclude&output=xml&count=1
19
ALEXANDRIA REAL ESTATE EQUITIES INC--1035443--20170131--0001035443-17-000022.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035688&type=10-K&dateb=20170223&owner=exclude&output=xml&count=1
20
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035748&type=10-K&dateb=20170227&owner=exclude&output=xml&count=1
5
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035976&type=10-K&dateb=20170310&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1035983&type=10-K&dateb=20170223&owner=exclude&output=xml&c

...

21
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1657853&type=10-K&dateb=20170306&owner=exclude&output=xml&count=1
28
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1657889&type=10-K&dateb=20170331&owner=exclude&output=xml&count=1
4
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1658305&type=10-K&dateb=20170303&owner=exclude&output=xml&count=1
10
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1658566&type=10-K&dateb=20170323&owner=exclude&output=xml&count=1
38
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1658982&type=10-K&dateb=20170330&owner=exclude&output=xml&count=1
18
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1659122&type=10-K&dateb=20170227&owner=exclude&output=xml&count=1
22
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1659166&type=10-K&dateb=20170228&owner=exclude&output=xml&count=1
24
Fortive Corp--1659166--20170228--0001659166-17-000091.txt  downloaded
http://www.

1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1667840&type=10-K&dateb=20170330&owner=exclude&output=xml&count=1
37
Best Hometown Bancorp Inc--1667840--20170330--0001571049-17-003036.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1667939&type=10-K&dateb=20170316&owner=exclude&output=xml&count=1
56
FSB Bancorp Inc--1667939--20170316--0001571049-17-002502.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1667944&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
26
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1667967&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1668186&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1668224&type=10-K&dateb=20170323&owner=exclude&output=xml&count=1
21
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&

1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1675192&type=10-K&dateb=20170329&owner=exclude&output=xml&count=1
4
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1675820&type=10-K&dateb=20170215&owner=exclude&output=xml&count=1
25
TiVo Corp--1675820--20170215--0001675820-17-000006.txt  downloaded
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1675887&type=10-K&dateb=20170323&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1675928&type=10-K&dateb=20170330&owner=exclude&output=xml&count=1
9
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1676479&type=10-K&dateb=20170313&owner=exclude&output=xml&count=1
7
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1676580&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
21
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1676667&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
21
http://www.sec.gov

HTTPConnectionPool(host='www.sec.gov', port=80): Max retries exceeded with url: /Archives/edgar/data/1684093/000119312517104230/0001193125-17-104230.txt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F57E0FF1D0>: Failed to establish a new connection: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다',))
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1684415&type=10-K&dateb=20170331&owner=exclude&output=xml&count=1
1
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1685022&type=10-K&dateb=20170320&owner=exclude&output=xml&count=1
21
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1685054&type=10-K&dateb=20170327&owner=exclude&output=xml&count=1
8
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1685212&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
3
http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1685715&type=10-K&dateb=20170324&owner=exclude&output=xml&count=1
1

ConnectionError: HTTPConnectionPool(host='www.sec.gov', port=80): Max retries exceeded with url: /cgi-bin/browse-edgar?action=getcompany&CIK=1687605&type=10-K&dateb=20170330&owner=exclude&output=xml&count=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F5063CFF98>: Failed to establish a new connection: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다',))